# Binned Training Notebook (Training-Only)

Train one model per cluster_Et bin using separate input files per bin. After training, compute per-bin AUC and the correlation between isoET and the model BDT score (predicted probability).

Bins: [5, 15, 25, 40] (GeV) — models trained on [5–15), [15–25), [25–40).

Outputs per bin:
- Trained model artifact (joblib)
- Metadata JSON
- Validation AUC
- isoET vs BDT score correlation (Pearson/Spearman)
- Plots: AUC by bin, isoET–BDT hexbin/scatter, optional ROC curves

In [1]:
# Section 1: Import Libraries and Set Seed
import os
import json
import time
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.ensemble import HistGradientBoostingClassifier

# Optional: XGBoost (preferred if available)
try:
    import xgboost as xgb
    XGB_AVAILABLE = True
except Exception:
    xgb = None
    XGB_AVAILABLE = False

from scipy.interpolate import UnivariateSpline
from scipy import stats
import joblib

import matplotlib.pyplot as plt
import seaborn as sns

# Reproducibility
GLOBAL_SEED = 42
np.random.seed(GLOBAL_SEED)

# Matplotlib defaults
plt.rcParams["figure.figsize"] = (7.5, 5)
plt.rcParams["axes.grid"] = True
sns.set_context("talk")

In [ ]:
# Section 2: Configure cluster_Et Bins and Per-Bin Input Files

# Reweighting configuration
# if CLASS_REWEIGHT: make the weighted signal and background the same
CLASS_REWEIGHT = True

# --- NEW: Option to train a single model across all Et bins ---
# If True, one model is trained on all data. Plots are still per-bin.
# If False, one model is trained for each bin.
TRAIN_SINGLE_MODEL = False

# Define bin edges and derive labels for evaluation
BIN_EDGES = [5, 15, 25, 40]
BIN_LABELS = [f"{BIN_EDGES[i]}_{BIN_EDGES[i+1]}" for i in range(len(BIN_EDGES)-1)]

# --- Input File Configuration ---
# Option 1: Use the same files for all bins (current setup)
# The data loader will filter events by pT for each bin.
# This is useful when you have inclusive data files.
USE_SINGLE_FILE_SET = False

SINGLE_FILE_SET = {
    "signal": "shapes_photon20.txt",
    "background": "shapes_jet30.txt"
}

# Option 2: Use different files for each pT bin.
# This is useful if your data is already split into separate files by pT.
# To use this, set USE_SINGLE_FILE_SET = False and edit the paths below.
PER_BIN_FILE_SETS = {
    "5_15": {
        "signal": "shapes_photon5.txt",      # Example path
        "background": "shapes_jet15.txt"     # Example path
    },
    "15_25": {
        "signal": "shapes_photon10.txt",     # Example path
        "background": "shapes_jet20.txt"    # Example path
    },
    "25_40": {
        "signal": "shapes_photon20.txt",     # Example path
        "background": "shapes_jet30.txt"    # Example path
    },
}

# Logic to select the configuration
if USE_SINGLE_FILE_SET:
    PER_BIN_FILES = {bin_label: SINGLE_FILE_SET for bin_label in BIN_LABELS}
else:
    PER_BIN_FILES = PER_BIN_FILE_SETS


# Column names from model_training.ipynb
COLUMNS = [
    "cluster_Et", "cluster_Eta", "cluster_Phi", "vertexz",
    "e11_over_e33", "e32_over_e35", "e11_over_e22", "e11_over_e13",
    "e11_over_e15", "e11_over_e17", "e11_over_e31",
    "e11_over_e51", "e11_over_e71", "e22_over_e33",
    "e22_over_e35", "e22_over_e37", "e22_over_e53",
    "cluster_prob", "cluster_weta_cogx", "cluster_wphi_cogx",
    "cluster_et1", "cluster_et2", "cluster_et3", "cluster_et4",
    "cluster_w32", "cluster_w52", "cluster_w72", 
    "recoisoET", "is_tight", "pid"
]

# Key column names
PT_COL = "cluster_Et"
ISO_COL = "recoisoET"      # Updated from "isoET"
LABEL_COL = "label"

# Feature columns to train on from model_training.ipynb
FEATURES = [
    "vertexz",
    "cluster_Eta",
    "e11_over_e33",
    "cluster_et1",
    "cluster_et2",
    "cluster_et3",
    "cluster_et4",
]

# Output directory for models/metrics
OUT_DIR = Path("binned_models")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Train/val split
TRAIN_SIZE = 0.8
VAL_SIZE = 1 - TRAIN_SIZE
STRATIFY = True

In [10]:
# Section 3: Load and Concatenate Data for Each Bin
try:
    import uproot  # for ROOT files
except Exception:
    uproot = None


def load_single(path: str, names: list) -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext in [".csv", ".txt"]:
        return pd.read_csv(path, sep=r"\s+", header=0, names=names)
    if ext in [".pq", ".parquet"]:
        return pd.read_parquet(path)
    if ext in [".root"]:
        if uproot is None:
            raise ImportError("uproot is required to read ROOT files. pip install uproot")
        # Heuristic: read first tree and all branches
        with uproot.open(path) as f:
            # Pick the first TTree-like object
            trees = [k for k in f.keys() if isinstance(f[k], uproot.behaviors.TTree.TTree)]
            if not trees:
                # Fall back: search members
                trees = [k for k, v in f.items() if hasattr(v, "arrays")]
            key = trees[0] if trees else list(f.keys())[0]
            arrs = f[key].arrays(library="pd")
            return arrs
    raise ValueError(f"Unsupported file type: {ext}")


def load_bin_data(bin_label: str, et_min: float, et_max: float) -> pd.DataFrame:
    file_map = PER_BIN_FILES.get(bin_label, {})
    if not file_map:
        raise FileNotFoundError(f"No input files configured for bin {bin_label}")

    dfs = []
    for file_type, path in file_map.items():
        if not os.path.exists(path):
            raise FileNotFoundError(f"Configured file not found: {path}")
        df_i = load_single(path, names=COLUMNS)
        if file_type == "signal":
            df_i[LABEL_COL] = 1
            df_i = df_i[df_i["pid"].isin([1,2])] # photon only
        elif file_type == "background":
            df_i[LABEL_COL] = 0
            df_i = df_i[~df_i["pid"].isin([1,2])] # reject electrons
        dfs.append(df_i)
    df = pd.concat(dfs, ignore_index=True)

    # Validate required columns
    base_required = {PT_COL, ISO_COL, LABEL_COL}
    if FEATURES:
        required = base_required.union(FEATURES)
    else:
        required = base_required
    missing = sorted(required - set(df.columns))
    if missing:
        raise KeyError(f"Missing required columns for bin {bin_label}: {missing}")

    # Drop rows with missing label and key columns
    df = df.dropna(subset=[LABEL_COL])

    # Filter by Et bin
    df = df[(df[PT_COL] >= et_min) & (df[PT_COL] < et_max)]

    # Ensure numeric dtypes for key columns
    for c in [PT_COL, ISO_COL, LABEL_COL]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    # Drop rows with NA on key columns after coercion
    df = df.dropna(subset=[PT_COL, ISO_COL, LABEL_COL])

    return df

# Section 3.1: Auto-detect FEATURES if empty (numeric columns excluding PT/ISO/LABEL)

def autodetect_features(df: pd.DataFrame) -> list:
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    exclude = {PT_COL, ISO_COL, LABEL_COL}
    feats = [c for c in numeric_cols if c not in exclude]
    return feats


def split_train_val_with_features(df: pd.DataFrame, features: list):
    X = df[features].copy()
    y = df[LABEL_COL].astype(int).values
    w = df["weight"].to_numpy(dtype=np.float32)
    
    strat = y if STRATIFY else None
    
    X_train, X_val, y_train, y_val, w_train, w_val = train_test_split(
        X, y, w, train_size=TRAIN_SIZE, random_state=GLOBAL_SEED, stratify=strat
    )
    return X_train, X_val, y_train, y_val, w_train, w_val

In [11]:
# Section 3.5: Kinematic Reweighting (Eta)

def apply_eta_reweighting(df: pd.DataFrame) -> pd.DataFrame:
    """
    Applies inverse-PDF reweighting to flatten the cluster_Eta distribution
    for signal and background separately.
    """
    df_out = df.copy()
    eta_min, eta_max = -0.7, 0.7
    n_bins_eta = 20
    bins_eta = np.linspace(eta_min, eta_max, n_bins_eta + 1)
    
    # --- Class Imbalance Reweighting ---
    if CLASS_REWEIGHT:
        n_sig = (df_out[LABEL_COL] == 1).sum()
        n_bkg = (df_out[LABEL_COL] == 0).sum()

        if n_sig > 0 and n_bkg > 0:
            weight_sig = (n_sig + n_bkg) / (2.0 * n_sig)
            weight_bkg = (n_sig + n_bkg) / (2.0 * n_bkg)
            df_out['class_weight'] = np.where(df_out[LABEL_COL] == 1, weight_sig, weight_bkg)
        else:
            df_out['class_weight'] = 1.0
        
    df_out["weight"] = df_out['class_weight']


    for label in [0, 1]:
        mask = df_out[LABEL_COL] == label
        if not mask.any():
            continue
            
        eta_vals = df_out.loc[mask, "cluster_Eta"].values

        # Histogram for eta PDF
        hist_eta, bin_edges_eta = np.histogram(eta_vals, bins=bins_eta, density=True)
        x_centers_eta = 0.5 * (bin_edges_eta[:-1] + bin_edges_eta[1:])

        # Fit spline to eta PDF
        spline_eta = UnivariateSpline(x_centers_eta, hist_eta, s=0.0)
        pdf_eta_vals = spline_eta(eta_vals)
        pdf_eta_vals = np.clip(pdf_eta_vals, a_min=1e-3, a_max=None)

        # Compute inverse PDF weights for eta
        weights_eta = 1.0 / pdf_eta_vals
        weights_eta /= np.mean(weights_eta)

        # Combine with class imbalance weight
        df_out.loc[mask, "weight"] *= weights_eta
        
    return df_out

In [12]:
# Section 3.6: QA Plots for Reweighting

def plot_reweighting_qa(df_before: pd.DataFrame, df_after: pd.DataFrame, bin_label: str):
    """
    Plots the cluster_Eta distribution and class balance before and after reweighting.
    """
    fig = plt.figure(figsize=(14, 10))
    gs = fig.add_gridspec(2, 2)

    # --- Eta Before Reweighting ---
    ax1 = fig.add_subplot(gs[0, 0])
    for label_val, name in zip([0, 1], ["Background", "Signal"]):
        mask = df_before[LABEL_COL] == label_val
        if mask.any():
            ax1.hist(df_before.loc[mask, "cluster_Eta"], bins=40, range=(-0.7, 0.7), 
                     histtype='step', label=name, density=True)
    ax1.set_title(f"Bin {bin_label}: Eta Before Reweighting")
    ax1.set_xlabel("cluster_Eta")
    ax1.set_ylabel("Density")
    ax1.legend()
    ax1.grid(True)

    # --- Eta After Reweighting ---
    ax2 = fig.add_subplot(gs[0, 1])
    for label_val, name in zip([0, 1], ["Background", "Signal"]):
        mask = df_after[LABEL_COL] == label_val
        if mask.any():
            ax2.hist(df_after.loc[mask, "cluster_Eta"], bins=40, range=(-0.7, 0.7),
                     weights=df_after.loc[mask, "weight"],
                     histtype='step', label=name, density=True)
    ax2.set_title(f"Bin {bin_label}: Eta After Reweighting")
    ax2.set_xlabel("cluster_Eta")
    ax2.legend()
    ax2.grid(True)

    # --- Class Balance Before ---
    ax3 = fig.add_subplot(gs[1, 0])
    counts_before = df_before[LABEL_COL].value_counts()
    n_sig_before = counts_before.get(1, 0)
    n_bkg_before = counts_before.get(0, 0)
    ax3.bar(["Background", "Signal"], [n_bkg_before, n_sig_before], color=['blue', 'orange'])
    ax3.set_title("Class Counts Before Reweighting")
    ax3.set_ylabel("Number of Events")
    ax3.grid(axis='y')
    for i, v in enumerate([n_bkg_before, n_sig_before]):
        ax3.text(i, v, str(v), ha='center', va='bottom')


    # --- Class Balance After ---
    ax4 = fig.add_subplot(gs[1, 1])
    w_sig_after = df_after.loc[df_after[LABEL_COL] == 1, "weight"].sum()
    w_bkg_after = df_after.loc[df_after[LABEL_COL] == 0, "weight"].sum()
    ax4.bar(["Background", "Signal"], [w_bkg_after, w_sig_after], color=['blue', 'orange'])
    ax4.set_title("Class Yield After Reweighting")
    ax4.set_ylabel("Sum of Weights")
    ax4.grid(axis='y')
    for i, v in enumerate([w_bkg_after, w_sig_after]):
        ax4.text(i, v, f"{v:.1f}", ha='center', va='bottom')

    
    plt.tight_layout()
    plt.show()

In [13]:
# Section 4: Train/Validation Split per Bin

def split_train_val(df: pd.DataFrame):
    X = df[FEATURES].copy()
    y = df[LABEL_COL].astype(int).values
    if STRATIFY:
        strat = y
    else:
        strat = None
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, train_size=TRAIN_SIZE, random_state=GLOBAL_SEED, stratify=strat
    )
    return X_train, X_val, y_train, y_val

In [14]:
# Section 5: Build Preprocessing and Model Pipeline

MODEL_CONFIG = {
    "classifier": "xgb" if 'XGB_AVAILABLE' in globals() and XGB_AVAILABLE else "hgb",
    "params": {
        # Defaults for XGBClassifier; if HGB used, some keys are ignored
        "n_estimators": 500,
        "max_depth": 5,
        "learning_rate": 0.05,
        "subsample": 0.8,
        "colsample_bytree": 0.8,
        "reg_alpha": 0.0,
        "reg_lambda": 1.0,
        "random_state": GLOBAL_SEED,
        "n_jobs": 4,
        "tree_method": "hist",
        #"scale_pos_weight": 10.0
    },
    "use_scaler": False,
}


def build_pipeline():
    steps = []
    steps.append(("imputer", SimpleImputer(strategy="median")))
    if MODEL_CONFIG.get("use_scaler", False):
        steps.append(("scaler", StandardScaler()))

    clf_name = MODEL_CONFIG["classifier"]
    if clf_name == "xgb" and XGB_AVAILABLE:
        clf = xgb.XGBClassifier(
            **MODEL_CONFIG["params"],
            objective="binary:logistic",
            eval_metric="auc",
            use_label_encoder=False,
        )
    elif clf_name == "hgb":
        hgb_params = {
            "max_depth": None if MODEL_CONFIG["params"].get("max_depth", 0) <= 0 else MODEL_CONFIG["params"]["max_depth"],
            "learning_rate": MODEL_CONFIG["params"].get("learning_rate", 0.1),
            "max_iter": MODEL_CONFIG["params"].get("n_estimators", 300),
            "l2_regularization": MODEL_CONFIG["params"].get("reg_lambda", 0.0),
            "min_samples_leaf": 20,
            "random_state": GLOBAL_SEED,
        }
        clf = HistGradientBoostingClassifier(**hgb_params)
    else:
        raise ValueError("Unsupported classifier in MODEL_CONFIG")

    steps.append(("clf", clf))
    return Pipeline(steps)

In [15]:
# Section 6: Train One Model per Bin

trained_pipelines = {}
train_metrics = {}

if TRAIN_SINGLE_MODEL:
    print("=== Training a single model for all bins ===")
    
    # 1. Load and combine data from all bins
    all_dfs_unweighted = []
    for i in range(len(BIN_EDGES) - 1):
        et_min, et_max = BIN_EDGES[i], BIN_EDGES[i+1]
        bin_label = BIN_LABELS[i]
        print(f"Loading data for bin {bin_label}...")
        df_bin_unweighted = load_bin_data(bin_label, et_min, et_max)
        all_dfs_unweighted.append(df_bin_unweighted)
    
    df_all_unweighted = pd.concat(all_dfs_unweighted, ignore_index=True)
    
    # 2. Apply reweighting to the combined dataset
    df_all = apply_eta_reweighting(df_all_unweighted)
    
    # 3. Get features and split into train/validation
    feats = FEATURES if FEATURES else autodetect_features(df_all)
    if not feats:
        raise ValueError("No FEATURES found for combined dataset. Please define FEATURES explicitly.")
        
    X_train, X_val, y_train, y_val, w_train, w_val = split_train_val_with_features(df_all, feats)

    # 4. Build and train the single pipeline
    pipe = build_pipeline()
    t0 = time.time()
    #if "sample_weight" in pipe.named_steps['clf'].fit.__code__.co_varnames:
    pipe.fit(X_train, y_train, clf__sample_weight=w_train)
    #else:
    #    pipe.fit(X_train, y_train)
    t1 = time.time()

    # 5. Calculate training AUC
    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        y_train_proba = pipe.predict_proba(X_train)[:, 1]
    else:
        y_train_proba = pipe.decision_function(X_train)
    auc_train = roc_auc_score(y_train, y_train_proba, sample_weight=w_train) if len(np.unique(y_train)) > 1 else np.nan

    # 6. Store results for each bin for evaluation
    df_val_all = df_all.loc[X_val.index]
    
    for i in range(len(BIN_EDGES) - 1):
        et_min, et_max = BIN_EDGES[i], BIN_EDGES[i+1]
        bin_label = BIN_LABELS[i]
        
        # Filter validation data for this bin
        val_mask = (df_val_all[PT_COL] >= et_min) & (df_val_all[PT_COL] < et_max)
        df_val_bin = df_val_all[val_mask]
        
        if df_val_bin.empty:
            continue

        X_val_bin = df_val_bin[feats]
        y_val_bin = df_val_bin[LABEL_COL].astype(int).values
        w_val_bin = df_val_bin["weight"].to_numpy(dtype=np.float32)

        trained_pipelines[bin_label] = {
            "pipeline": pipe,  # The single trained pipeline
            "features": feats,
            "X_val": X_val_bin,
            "y_val": y_val_bin,
            "w_val": w_val_bin,
            "val_index": X_val_bin.index,
            "df_bin": df_all,  # Full dataframe for ISO correlation
            "df_bin_unweighted": df_all_unweighted, # Full unweighted for QA
        }
        train_metrics[bin_label] = {
            "train_time_sec": round(t1 - t0, 3),
            "n_train": int(len(y_train)),
            "n_val": int(len(y_val_bin)),
            "auc_train": float(auc_train) if not np.isnan(auc_train) else np.nan,
            "et_min": float(et_min),
            "et_max": float(et_max),
        }

else:
    # Original per-bin training loop
    for i in range(len(BIN_EDGES) - 1):
        et_min, et_max = BIN_EDGES[i], BIN_EDGES[i+1]
        bin_label = BIN_LABELS[i]
        print(f"\n=== Training bin {bin_label} [{et_min}, {et_max}) GeV ===")

        df_bin_unweighted = load_bin_data(bin_label, et_min, et_max)
        
        # Apply reweighting
        df_bin = apply_eta_reweighting(df_bin_unweighted)
        
        feats = FEATURES if FEATURES else autodetect_features(df_bin)
        if not feats:
            raise ValueError(f"No FEATURES found for bin {bin_label}. Please define FEATURES explicitly.")

        X_train, X_val, y_train, y_val, w_train, w_val = split_train_val_with_features(df_bin, feats)

        print(w_train)

        pipe = build_pipeline()
        t0 = time.time()
        
        # Pass sample weights to the fit method
        #if "sample_weight" in pipe.named_steps['clf'].fit.__code__.co_varnames:
        print("Train with sample weights")
        pipe.fit(X_train, y_train, clf__sample_weight=w_train)
            
        #else:
        #    pipe.fit(X_train, y_train)
            
        t1 = time.time()

        # Basic training AUC on train split
        if hasattr(pipe.named_steps["clf"], "predict_proba"):
            y_train_proba = pipe.predict_proba(X_train)[:, 1]
        else:
            y_train_proba = pipe.decision_function(X_train)
        auc_train = roc_auc_score(y_train, y_train_proba, sample_weight=w_train) if len(np.unique(y_train)) > 1 else np.nan

        trained_pipelines[bin_label] = {
            "pipeline": pipe,
            "features": feats,
            "X_val": X_val,
            "y_val": y_val,
            "w_val": w_val,
            "val_index": X_val.index,  # store for alignment
            "df_bin": df_bin,  # keep for ISO correlation reference
            "df_bin_unweighted": df_bin_unweighted, # For QA plots
        }
        train_metrics[bin_label] = {
            "train_time_sec": round(t1 - t0, 3),
            "n_train": int(len(y_train)),
            "n_val": int(len(y_val)),
            "auc_train": float(auc_train) if not np.isnan(auc_train) else np.nan,
            "et_min": float(et_min),
            "et_max": float(et_max),
        }

print("\nTraining complete for all bins.")


=== Training bin 5_15 [5, 15) GeV ===


FileNotFoundError: Configured file not found: shapes_photon_10.txt

In [ ]:
# Section 7: Persist Trained Pipelines and Metadata

if TRAIN_SINGLE_MODEL and trained_pipelines:
    # Save the single model once
    first_bin = next(iter(trained_pipelines.keys()))
    pipe = trained_pipelines[first_bin]["pipeline"]
    feats = trained_pipelines[first_bin]["features"]
    
    model_path = OUT_DIR / "model_single.joblib"
    joblib.dump(pipe, model_path)

    # Save metadata for each bin, referencing the single model
    for bin_label in BIN_LABELS:
        if bin_label not in trained_pipelines:
            continue
        meta = {
            "bin_label": bin_label,
            "trained_as_single_model": True,
            "single_model_path": str(model_path),
            "bin_edges": BIN_EDGES,
            "pt_col": PT_COL,
            "iso_col": ISO_COL,
            "label_col": LABEL_COL,
            "features": feats,
            "train_size": TRAIN_SIZE,
            "random_seed": GLOBAL_SEED,
            "model_config": MODEL_CONFIG,
            "train_metrics": train_metrics.get(bin_label, {}),
            "input_files": PER_BIN_FILES.get(bin_label, []),
        }
        with open(OUT_DIR / f"metadata_{bin_label}.json", "w") as f:
            json.dump(meta, f, indent=2)
else:
    # Original logic for per-bin models
    for bin_label in BIN_LABELS:
        if bin_label not in trained_pipelines:
            continue
        entry = trained_pipelines[bin_label]
        pipe = entry["pipeline"]
        feats = entry["features"]

        model_path = OUT_DIR / f"model_{bin_label}.joblib"
        joblib.dump(pipe, model_path)

        meta = {
            "bin_label": bin_label,
            "trained_as_single_model": False,
            "bin_edges": BIN_EDGES,
            "pt_col": PT_COL,
            "iso_col": ISO_COL,
            "label_col": LABEL_COL,
            "features": feats,
            "train_size": TRAIN_SIZE,
            "random_seed": GLOBAL_SEED,
            "model_config": MODEL_CONFIG,
            "train_metrics": train_metrics.get(bin_label, {}),
            "input_files": PER_BIN_FILES.get(bin_label, []),
        }
        with open(OUT_DIR / f"metadata_{bin_label}.json", "w") as f:
            json.dump(meta, f, indent=2)

print(f"Saved models and metadata to: {OUT_DIR}")

In [ ]:
# Section 6.5: Reweighting QA Plots

if TRAIN_SINGLE_MODEL and trained_pipelines:
    # For a single model, show QA for the combined dataset
    first_bin = next(iter(trained_pipelines.keys()))
    df_unweighted = trained_pipelines[first_bin]["df_bin_unweighted"]
    df_weighted = trained_pipelines[first_bin]["df_bin"]
    plot_reweighting_qa(df_unweighted, df_weighted, "all_bins_combined")
else:
    # Original per-bin QA plots
    for bin_label in BIN_LABELS:
        if bin_label in trained_pipelines:
            df_unweighted = trained_pipelines[bin_label]["df_bin_unweighted"]
            df_weighted = trained_pipelines[bin_label]["df_bin"]
            plot_reweighting_qa(df_unweighted, df_weighted, bin_label)

In [ ]:
# Section 8: Evaluate AUC per pT/cluster_Et Bin

val_results = {}

for bin_label in BIN_LABELS:
    if bin_label not in trained_pipelines:
        continue
    entry = trained_pipelines[bin_label]
    pipe = entry["pipeline"]
    X_val, y_val, w_val = entry["X_val"], entry["y_val"], entry["w_val"]

    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        y_proba = pipe.predict_proba(X_val)[:, 1]
    else:
        y_proba = pipe.decision_function(X_val)

    auc = roc_auc_score(y_val, y_proba, sample_weight=w_val) if len(np.unique(y_val)) > 1 else np.nan
    fpr, tpr, thr = roc_curve(y_val, y_proba, sample_weight=w_val) if len(np.unique(y_val)) > 1 else (None, None, None)

    val_results[bin_label] = {
        "auc_val": float(auc) if not np.isnan(auc) else np.nan,
        "fpr": fpr,
        "tpr": tpr,
        "thresholds": thr,
        "n_val": int(len(y_val)),
    }

val_results

In [ ]:
# Section 9: Compute isoET vs BDT Score Correlation per Bin

correlation_results = {}

for bin_label in BIN_LABELS:
    if bin_label not in trained_pipelines:
        continue
    entry = trained_pipelines[bin_label]
    pipe = entry["pipeline"]
    X_val, y_val, df_bin, val_index, w_val = entry["X_val"], entry["y_val"], entry["df_bin"], entry["val_index"], entry["w_val"]

    # isoET values for the validation rows
    iso_val = df_bin.loc[val_index, ISO_COL].to_numpy()

    # Predicted prob on validation set
    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        y_proba = pipe.predict_proba(X_val)[:, 1]
    else:
        y_proba = pipe.decision_function(X_val)

    # Drop any NaNs
    mask = np.isfinite(iso_val) & np.isfinite(y_proba) & np.isfinite(w_val)
    iso_c = iso_val[mask]
    proba_c = y_proba[mask]
    w_c = w_val[mask]

    # Weighted Pearson correlation
    def weighted_pearson(x, y, w):
        mx = np.average(x, weights=w)
        my = np.average(y, weights=w)
        cov = np.average((x - mx) * (y - my), weights=w)
        sx = np.sqrt(np.average((x - mx)**2, weights=w))
        sy = np.sqrt(np.average((y - my)**2, weights=w))
        if sx * sy == 0:
            return np.nan
        return cov / (sx * sy)

    pearson_r = weighted_pearson(iso_c, proba_c, w_c) if len(iso_c) > 1 else np.nan
    # Spearman correlation is rank-based and doesn't have a standard weighted version
    spearman_rho, spearman_p = stats.spearmanr(iso_c, proba_c) if len(iso_c) > 1 else (np.nan, np.nan)

    correlation_results[bin_label] = {
        "pearson_r": float(pearson_r) if np.isfinite(pearson_r) else np.nan,
        "pearson_p": np.nan, # p-value for weighted correlation is complex
        "spearman_rho": float(spearman_rho) if np.isfinite(spearman_rho) else np.nan,
        "spearman_p": float(spearman_p) if np.isfinite(spearman_p) else np.nan,
        "n_pairs": int(np.sum(mask)),
    }

correlation_results

In [ ]:
# Section 10: Visualize AUC and isoET–BDT Relationships

# --- Helper function for profile plots ---
def plot_feature_profile(df_bin, val_index, y_proba, feature_name, bin_label, w_val):
    """
    Generates a 2D histogram and profile plot for a given feature vs. BDT score.
    - Calculates and annotates the weighted Pearson correlation between the feature and BDT score.
    - Plots the feature on the y-axis except when the feature is isoET (ISO_COL); in that case, BDT score is on the y-axis.
    """
    
    # Separate signal and background in the validation set
    val_df = df_bin.loc[val_index].copy()
    val_df['bdt_score'] = y_proba
    val_df['weight'] = w_val

    sig_df = val_df[val_df[LABEL_COL] == 1]
    bkg_df = val_df[val_df[LABEL_COL] == 0]

    # Axis rule: feature on y-axis, except for isoET -> swap axes
    invert_axes = not (feature_name == ISO_COL)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6), sharey=invert_axes)
    title_xy = f"BDT Score vs. {feature_name}" if invert_axes else f"{feature_name} vs. BDT Score"
    fig.suptitle(f"Profile of {title_xy} (Bin: {bin_label})", fontsize=16)

    score_bins = np.linspace(0, 1, 41)

    # Weighted Pearson correlation helper
    def weighted_pearson(x, y, w):
        mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(w)
        x, y, w = x[mask], y[mask], w[mask]
        if x.size < 2:
            return np.nan
        w_sum = np.sum(w)
        mx = np.sum(w * x) / w_sum
        my = np.sum(w * y) / w_sum
        cov = np.sum(w * (x - mx) * (y - my)) / w_sum
        sx = np.sqrt(np.sum(w * (x - mx) ** 2) / w_sum)
        sy = np.sqrt(np.sum(w * (y - my) ** 2) / w_sum)
        if sx == 0 or sy == 0:
            return np.nan
        return cov / (sx * sy)

    # --- Signal Plot ---
    if invert_axes:
        x_sig = sig_df[feature_name]
        y_sig = sig_df['bdt_score']
        w_sig = sig_df['weight']
        # Determine feature binning for x-axis
        if len(x_sig) > 0:
            x_min, x_max = np.nanpercentile(x_sig, [1, 99])
            if not np.isfinite(x_min) or not np.isfinite(x_max) or x_min == x_max:
                x_min, x_max = np.nanmin(x_sig), np.nanmax(x_sig)
        else:
            x_min, x_max = 0.0, 1.0
        feat_bins = np.linspace(x_min, x_max, 51)
        ax1.hist2d(x_sig, y_sig, bins=[feat_bins, score_bins], cmin=1, weights=w_sig, cmap='viridis')
        # Profile (mean of y in x-bins)
        bin_centers = 0.5 * (feat_bins[:-1] + feat_bins[1:])
        bin_means, _, _ = stats.binned_statistic(x_sig, y_sig, statistic='mean', bins=feat_bins)
        ax1.plot(bin_centers, bin_means, 'r-o', lw=2, label='Profile')
        ax1.set_xlabel(feature_name)
        ax1.set_ylabel("BDT Score")
    else:
        x_sig = sig_df['bdt_score']
        y_sig = sig_df[feature_name]
        w_sig = sig_df['weight']
        ax1.hist2d(x_sig, y_sig, bins=[score_bins, 50], cmin=1, weights=w_sig, cmap='viridis')
        # Profile (mean of y in BDT-score bins)
        bin_centers = (score_bins[:-1] + score_bins[1:]) / 2
        bin_means, _, _ = stats.binned_statistic(x_sig, y_sig, statistic='mean', bins=score_bins)
        ax1.plot(bin_centers, bin_means, 'r-o', lw=2, label='Profile')
        ax1.set_xlabel("BDT Score")
        ax1.set_ylabel(feature_name)

    r_sig = weighted_pearson(x_sig, y_sig, w_sig)
    ax1.set_title("Signal")
    ax1.legend()
    ax1.grid(True)
    ax1.text(0.02, 0.98, f"r = {r_sig:.3f}", transform=ax1.transAxes, ha='left', va='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

    # --- Background Plot ---
    if invert_axes:
        x_bkg = bkg_df[feature_name]
        y_bkg = bkg_df['bdt_score']
        w_bkg = bkg_df['weight']
        # Reuse feat_bins where possible; recompute if empty
        if len(x_bkg) > 0 and np.isfinite(np.nanmin(x_bkg)) and np.isfinite(np.nanmax(x_bkg)):
            ax2.hist2d(x_bkg, y_bkg, bins=[feat_bins, score_bins], cmin=1, weights=w_bkg, cmap='viridis')
            bin_centers_bkg = 0.5 * (feat_bins[:-1] + feat_bins[1:])
            bin_means_bkg, _, _ = stats.binned_statistic(x_bkg, y_bkg, statistic='mean', bins=feat_bins)
        else:
            ax2.hist2d(x_bkg, y_bkg, bins=[50, score_bins], cmin=1, weights=w_bkg, cmap='viridis')
            # approximate centers from data range
            xb_min, xb_max = (np.nanmin(x_bkg) if len(x_bkg) else 0.0), (np.nanmax(x_bkg) if len(x_bkg) else 1.0)
            feat_bins_bkg = np.linspace(xb_min, xb_max, 51)
            bin_centers_bkg = 0.5 * (feat_bins_bkg[:-1] + feat_bins_bkg[1:])
            bin_means_bkg, _, _ = stats.binned_statistic(x_bkg, y_bkg, statistic='mean', bins=feat_bins_bkg)
        ax2.plot(bin_centers_bkg, bin_means_bkg, 'r-o', lw=2, label='Profile')
        ax2.set_xlabel(feature_name)
        ax2.set_ylabel("BDT Score")
    else:
        x_bkg = bkg_df['bdt_score']
        y_bkg = bkg_df[feature_name]
        w_bkg = bkg_df['weight']
        ax2.hist2d(x_bkg, y_bkg, bins=[score_bins, 50], cmin=1, weights=w_bkg, cmap='viridis')
        bin_centers_bkg = (score_bins[:-1] + score_bins[1:]) / 2
        bin_means_bkg, _, _ = stats.binned_statistic(x_bkg, y_bkg, statistic='mean', bins=score_bins)
        ax2.plot(bin_centers_bkg, bin_means_bkg, 'r-o', lw=2, label='Profile')
        ax2.set_xlabel("BDT Score")
        ax2.set_ylabel(feature_name)

    r_bkg = weighted_pearson(x_bkg, y_bkg, w_bkg)
    ax2.set_title("Background")
    ax2.legend()
    ax2.grid(True)
    ax2.text(0.02, 0.98, f"r = {r_bkg:.3f}", transform=ax2.transAxes, ha='left', va='top',
             bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

# --- Overlayed profile plots across bins ---
def overlay_feature_profiles(trained_pipelines, feature_name, ISO_COL, LABEL_COL, BIN_LABELS):
    """
    Draw profile curves overlaid for all bins:
      - Left: signal profiles
      - Right: background profiles
    Axis rule: feature on x-axis, BDT score on y-axis,
               EXCEPT when feature is isoET (ISO_COL) -> swap axes.
    """
    # Collect per-bin arrays to define a common x-binning
    xs_sig_all, xs_bkg_all = [], []
    profiles_sig, profiles_bkg = {} , {}

    invert_axes = not (feature_name == ISO_COL)  # same rule as your helper
    score_bins = np.linspace(0, 1, 41)           # used when x is BDT score

    # Pass 1: gather all x values per class for global bin edges
    for bin_label in BIN_LABELS:
        if bin_label not in trained_pipelines:
            continue
        entry = trained_pipelines[bin_label]
        pipe = entry["pipeline"]
        X_val = entry["X_val"]
        df_bin = entry["df_bin"].loc[entry["val_index"]].copy()
        w_val = entry["w_val"]

        # get scores
        if hasattr(pipe.named_steps["clf"], "predict_proba"):
            y_proba = pipe.predict_proba(X_val)[:, 1]
        else:
            y_proba = pipe.decision_function(X_val)

        df_bin["bdt_score"] = y_proba
        df_bin["weight"] = w_val

        sig = df_bin[df_bin[LABEL_COL] == 1]
        bkg = df_bin[df_bin[LABEL_COL] == 0]

        if invert_axes:
            # x = feature, y = score
            xs_sig_all.append(sig[feature_name].to_numpy())
            xs_bkg_all.append(bkg[feature_name].to_numpy())
        else:
            # x = score, y = feature
            xs_sig_all.append(sig["bdt_score"].to_numpy())
            xs_bkg_all.append(bkg["bdt_score"].to_numpy())

    # Define common x-bins per class (robust to outliers)
    def robust_bins(arr_list, nbins=50):
        if len(arr_list) == 0:
            return np.linspace(0, 1, nbins+1)
        x_all = np.concatenate([a[np.isfinite(a)] for a in arr_list if a is not None])
        if x_all.size == 0:
            return np.linspace(0, 1, nbins+1)
        x1, x99 = np.nanpercentile(x_all, [1, 99])
        if not np.isfinite(x1) or not np.isfinite(x99) or x1 == x99:
            x1, x99 = np.nanmin(x_all), np.nanmax(x_all)
            if x1 == x99:
                x1, x99 = x1 - 0.5, x1 + 0.5
        return np.linspace(x1, x99, nbins+1)

    if invert_axes:
        # feature on x for both panels
        xbins_sig = robust_bins(xs_sig_all, nbins=50)
        xbins_bkg = robust_bins(xs_bkg_all, nbins=50)
    else:
        # BDT score on x for both panels
        xbins_sig = score_bins
        xbins_bkg = score_bins

    # Helper: unweighted mean per x-bin (matches your current behavior)
    def binned_mean(x, y, bins):
        # returns bin centers, and mean of y in each x-bin
        inds = np.digitize(x, bins) - 1
        centers = 0.5 * (bins[:-1] + bins[1:])
        means = np.full(len(centers), np.nan)
        for bi in range(len(centers)):
            m = inds == bi
            if np.any(m):
                means[bi] = np.nanmean(y[m])
        return centers, means

    # Pass 2: compute profile per bin (signal & background)
    for bin_label in BIN_LABELS:
        if bin_label not in trained_pipelines:
            continue
        entry = trained_pipelines[bin_label]
        pipe = entry["pipeline"]
        X_val = entry["X_val"]
        df_bin = entry["df_bin"].loc[entry["val_index"]].copy()

        if hasattr(pipe.named_steps["clf"], "predict_proba"):
            y_proba = pipe.predict_proba(X_val)[:, 1]
        else:
            y_proba = pipe.decision_function(X_val)

        df_bin["bdt_score"] = y_proba

        sig = df_bin[df_bin[LABEL_COL] == 1]
        bkg = df_bin[df_bin[LABEL_COL] == 0]

        if invert_axes:
            x_s, y_s = sig[feature_name].to_numpy(), sig["bdt_score"].to_numpy()
            x_b, y_b = bkg[feature_name].to_numpy(), bkg["bdt_score"].to_numpy()
        else:
            x_s, y_s = sig["bdt_score"].to_numpy(), sig[feature_name].to_numpy()
            x_b, y_b = bkg["bdt_score"].to_numpy(), bkg[feature_name].to_numpy()

        cs, ms = binned_mean(x_s, y_s, xbins_sig)
        cb, mb = binned_mean(x_b, y_b, xbins_bkg)

        profiles_sig[bin_label] = (cs, ms)
        profiles_bkg[bin_label] = (cb, mb)

    # Plot overlays
    fig, (axS, axB) = plt.subplots(1, 2, figsize=(16, 6), sharey=not invert_axes)
    if invert_axes:
        axS.set_xlabel(feature_name); axS.set_ylabel("BDT Score")
        axB.set_xlabel(feature_name); axB.set_ylabel("BDT Score")
        supt = f"Overlayed Profiles: BDT vs {feature_name}"
    else:
        axS.set_xlabel("BDT Score");   axS.set_ylabel(feature_name)
        axB.set_xlabel("BDT Score");   axB.set_ylabel(feature_name)
        supt = f"Overlayed Profiles: {feature_name} vs BDT"

    for bin_label, (c, m) in profiles_sig.items():
        axS.plot(c, m, marker='o', lw=2, label=str(bin_label))
    axS.set_title("Signal"); axS.grid(True); axS.legend()

    for bin_label, (c, m) in profiles_bkg.items():
        axB.plot(c, m, marker='o', lw=2, label=str(bin_label))
    axB.set_title("Background"); axB.grid(True); axB.legend()

    fig.suptitle(supt, fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

def plot_bdt_score_overlay_per_bin(trained_pipelines, BIN_LABELS, LABEL_COL, n_bins=50, density=True):
    """
    For each bin in BIN_LABELS, overlay the weighted BDT-score distributions of signal and background.
    - Uses per-bin common bin edges (computed from S∪B scores) so curves are comparable.
    - Normalizes to density by default (area=1 for each class in that bin), controlled by `density`.
    """
    for bin_label in BIN_LABELS:
        if bin_label not in trained_pipelines:
            continue
        entry = trained_pipelines[bin_label]
        pipe = entry["pipeline"]
        X_val = entry["X_val"]
        df_val = entry["df_bin"].loc[entry["val_index"]].copy()
        w_val = entry["w_val"]

        # Get BDT scores
        if hasattr(pipe.named_steps["clf"], "predict_proba"):
            scores = pipe.predict_proba(X_val)[:, 1]
            score_label = "BDT Score"
            # Prefer [0,1] bins for probabilities
            bin_edges = np.linspace(0, 1, n_bins + 1)
        else:
            scores = pipe.decision_function(X_val)
            score_label = "BDT Score (decision_function)"
            # Robust binning from combined S∪B (1–99% to avoid outliers)
            p1, p99 = np.nanpercentile(scores[np.isfinite(scores)], [1, 99])
            if not np.isfinite(p1) or not np.isfinite(p99) or p1 == p99:
                p1, p99 = np.nanmin(scores), np.nanmax(scores)
                if p1 == p99:
                    p1, p99 = p1 - 0.5, p1 + 0.5
            bin_edges = np.linspace(p1, p99, n_bins + 1)

        df_val["score"] = scores
        df_val["w"] = w_val

        sig = df_val[df_val[LABEL_COL] == 1]
        bkg = df_val[df_val[LABEL_COL] == 0]

        # Build the plot
        plt.figure(figsize=(7, 5))
        # Background first so signal can overlay clearly
        plt.hist(
            bkg["score"].values,
            bins=bin_edges,
            weights=bkg["w"].values,
            histtype="stepfilled",
            alpha=0.35,
            label="Background",
            density=density,
        )
        plt.hist(
            sig["score"].values,
            bins=bin_edges,
            weights=sig["w"].values,
            histtype="step",
            linewidth=2,
            label="Signal",
            density=density,
        )

        # Styling
        plt.title(f"BDT Score Distribution — Bin: {bin_label}")
        plt.xlabel(score_label)
        plt.ylabel("Density" if density else "Weighted counts")
        plt.grid(True, alpha=0.35)
        plt.legend()
        plt.tight_layout()
        plt.show()




In [ ]:

# --- AUC Bar Plot ---
auc_data = [(b, val_results[b]["auc_val"]) for b in BIN_LABELS if b in val_results]
if auc_data:
    bins_, aucs_ = zip(*auc_data)
    plt.figure()
    sns.barplot(x=list(bins_), y=list(aucs_), color="steelblue")
    plt.ylabel("Validation AUC")
    plt.xlabel("cluster_Et bin [GeV]")
    plt.title("AUC by pT bin")
    plt.ylim(0.5, 1.0)
    for i, v in enumerate(aucs_):
        plt.text(i, v + 0.01, f"{v:.3f}", ha="center")
    plt.show()

# --- Combined ROC Curves ---
plt.figure(figsize=(8, 8))
for bin_label in BIN_LABELS:
    if bin_label not in val_results:
        continue
    r = val_results[bin_label]
    if r["fpr"] is not None and r["tpr"] is not None:
        plt.plot(r["fpr"], r["tpr"], label=f"{bin_label} (AUC={r['auc_val']:.3f})")

plt.plot([0, 1], [0, 1], "k--", alpha=0.5, label="Chance")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curves for all pT Bins")
plt.legend()
plt.grid(True)
plt.axis('square')
plt.xlim(0, 1)
plt.ylim(0, 1)
plt.show()


In [ ]:


# --- Profile Plots for isoET and all Training Features ---
for bin_label in BIN_LABELS:
    if bin_label not in trained_pipelines:
        continue
    entry = trained_pipelines[bin_label]
    pipe = entry["pipeline"]
    X_val = entry["X_val"]
    df_bin = entry["df_bin"]
    val_index = entry["val_index"]
    w_val = entry["w_val"]
    
    if hasattr(pipe.named_steps["clf"], "predict_proba"):
        y_proba = pipe.predict_proba(X_val)[:, 1]
    else:
        y_proba = pipe.decision_function(X_val)

    # Plot for recoisoET
    plot_feature_profile(df_bin, val_index, y_proba, ISO_COL, bin_label, w_val)
    
    # Plot for cluster_Et
    plot_feature_profile(df_bin, val_index, y_proba, PT_COL, bin_label, w_val)

    # Plots for all training features
    for feature in entry["features"]:
        plot_feature_profile(df_bin, val_index, y_proba, feature, bin_label, w_val)
for feature in trained_pipelines[next(iter(trained_pipelines))]["features"]:
    overlay_feature_profiles(trained_pipelines, feature, ISO_COL, LABEL_COL, BIN_LABELS)
overlay_feature_profiles(trained_pipelines, PT_COL, ISO_COL, LABEL_COL, BIN_LABELS)


In [ ]:

plot_bdt_score_overlay_per_bin(
    trained_pipelines=trained_pipelines,
    BIN_LABELS=BIN_LABELS,
    LABEL_COL=LABEL_COL,
    n_bins=50,       # tweak if you want smoother/rougher histograms
    density=True     # set False to see weighted counts instead
)

In [ ]:
# Section 11: Summarize and Save Metrics Table

rows = []
for bin_label in BIN_LABELS:
    row = {"bin": bin_label}
    # Training metrics
    row.update(train_metrics.get(bin_label, {}))
    # Validation AUC
    if bin_label in val_results:
        row["auc_val"] = val_results[bin_label]["auc_val"]
        row["n_val"] = val_results[bin_label]["n_val"]
    else:
        row["auc_val"] = np.nan
        row["n_val"] = 0
    # Correlations
    if bin_label in correlation_results:
        row.update(correlation_results[bin_label])
    else:
        row.update({
            "pearson_r": np.nan,
            "pearson_p": np.nan,
            "spearman_rho": np.nan,
            "spearman_p": np.nan,
            "n_pairs": 0,
        })
    rows.append(row)

metrics_df = pd.DataFrame(rows)
metrics_df = metrics_df[[
    "bin", "et_min", "et_max", "n_train", "n_val", "auc_train", "auc_val",
    "pearson_r", "pearson_p", "spearman_rho", "spearman_p", "n_pairs", "train_time_sec"
]].sort_values("bin")

metrics_path = OUT_DIR / "per_bin_metrics.csv"
metrics_df.to_csv(metrics_path, index=False)

print("Metrics saved:", metrics_path)
metrics_df